In [1]:
# Import all necessary libraries

import nltk
import numpy as np
import json
import glob

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary

import spacy
from nltk.corpus import stopwords


import pyLDAvis
import pyLDAvis.gensim

/Users/theomitchell/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [2]:
# Create a function to load our data from a json file
def load_data(file):
    with open(file, "r", encoding = "utf-8") as f:
        data = json.load(f)
    return (data)

In [3]:
# Create and Update a repository of stopwords

en = spacy.load('en_core_web_sm')
sw_spacy = en.Defaults.stop_words
sw_spacy.add('fire')
sw_spacy.add('wildfire')
sw_spacy.add('forest')
sw_spacy.add('ion')
sw_spacy.add('well')

330
{'are', 'her', 'your', 'namely', 'hereafter', 'no', 'becomes', 'behind', 'someone', 'i', 'less', 'some', 'beforehand', 'yours', 'fifteen', 'across', 'name', 'eleven', 'towards', 'will', 'were', 'nothing', 'down', 'nobody', 'wildfire', 'by', 'all', 'ourselves', 'take', 'not', 'n’t', 'seem', 'n‘t', 'ever', 'we', 'indeed', 'further', 'say', 'among', 'done', 'top', 'even', 'really', 'except', 'than', '‘s', 'twelve', "'ll", 'anything', 'themselves', 'either', 'whence', 'using', 'besides', 'at', 'everyone', 'anyway', 'now', 'each', 'though', 'eight', 'then', 'yourself', 'wherein', 'of', 'used', 'please', 'sixty', 'former', 'just', 'during', 'these', 'least', 'ca', 'but', 'and', 'himself', 'next', 'although', 'onto', 'another', 'you', 'before', 'beside', 'should', 'much', '’m', 'become', 'which', 'whole', 'with', 'since', 'here', 'seemed', 'none', 'there', 'twenty', 'had', 'neither', 'both', 'perhaps', 'moreover', 'around', 'their', 'sometimes', 'is', 'hence', 'our', 'formerly', 'hereupon

In [4]:
# load the data from all different files

USFS11 = load_data("Desktop/Json_files/USFS1.1.json")['page_data']
USFS12 = load_data("Desktop/Json_files/USFS1.2.json")['page_data']
USFS21 = load_data("Desktop/Json_files/USFS2.1.json")['page_data']
USFS22 = load_data("Desktop/Json_files/USFS2.2.json")['page_data']
USFS23 = load_data("Desktop/Json_files/USFS2.3.json")['page_data']
USFS31 = load_data("Desktop/Json_files/USFS3.1.json")['page_data']
USFS32 = load_data("Desktop/Json_files/USFS3.2.json")['page_data']
USFS33 = load_data("Desktop/Json_files/USFS3.3.json")['page_data']
USFS34 = load_data("Desktop/Json_files/USFS3.4.json")['page_data']
USFS35 = load_data("Desktop/Json_files/USFS3.5.json")['page_data']
USFS36 = load_data("Desktop/Json_files/USFS3.6.json")['page_data']
USFS37 = load_data("Desktop/Json_files/USFS3.7.json")['page_data']
USFS38 = load_data("Desktop/Json_files/USFS3.8.json")['page_data']
USFStext = []

In [5]:
# Create functions to merge our data into one list

def json_to_textlist(file):
    text_out = []
    for i in range(len(file)):
        text_out.append(file[i]['raw_text'])
    return text_out

def file_list_merge(file, l):
    for i in range(len(file)):
        l.append(file[i]['raw_text'])
    return l

In [6]:
# Merge data

USFStext = json_to_textlist(USFS11)
file_list_merge(USFS12, USFStext)
file_list_merge(USFS21, USFStext)
file_list_merge(USFS22, USFStext)
file_list_merge(USFS23, USFStext)
file_list_merge(USFS31, USFStext)
file_list_merge(USFS32, USFStext)
file_list_merge(USFS33, USFStext)
file_list_merge(USFS34, USFStext)
file_list_merge(USFS35, USFStext)
file_list_merge(USFS36, USFStext)
file_list_merge(USFS37, USFStext)
file_list_merge(USFS38, USFStext)

["Key Facts | Revised Fire , Management Policy Every revised wildfire policy does requires an Service allow appropriate for suppression action : The The Forest not a weakened fire protection will posture. wildfires continue to take immediate action to control I The initial attack policy has not changed. . continue Fire will be high energetic suppression to fast, s thorough , and conducted with a degree of regard for personal safety | Through the policy revision , the Chief has direction. replaced the rigid guided as ControlPolicy very known the 10AM s which has control of escaped fires since 1935. If fire initial escapes attack. 1 an escaped fire situation analysis will be made by the land effective manager | These situation analyses will f include cost- fire deciding suppression alternatives and will serve as the bas is for which alternative to use | Fire protection and use programs will be planned to be cost- effective and will provide protection for life and property A | public safe

In [7]:
#Create a lemmatization function that will turn all words into their root forms and eliminate stopwords, then lemmatize our data. 

def lemmatization(texts, allowed_postags = ["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable = ["parser", "ner"])
    texts_out = []
    for text in texts: 
        doc = nlp(text)
        new_text = []
        for token in doc: 
            if token.pos_ in allowed_postags and token.text not in sw_spacy:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return(texts_out)

lemmatized_USFS = lemmatization(USFStext)

In [8]:
# Create a list of each individual lemmatized word 

def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc = True)
        final.append(new)
    return (final)

USFS_clean = gen_words(lemmatized_USFS)

print(USFS_clean[0][0:20])

['key', 'fact', 'revise', 'policy', 'require', 'allow', 'appropriate', 'suppression', 'action', 'weakened', 'protection', 'posture', 'wildfire', 'continue', 'immediate', 'action', 'control', 'initial', 'attack', 'policy']


In [9]:
# Transform our list of individual words into a list of words and their frequency

id2word = corpora.Dictionary(USFS_clean)

corpus = []
for text in USFS_clean:
    new = id2word.doc2bow(text)
    corpus.append(new)
    

In [10]:
# Create the LDA Model

lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus, 
                                            id2word = id2word, 
                                            num_topics = 5, 
                                            random_state = 100, 
                                            update_every = 1,
                                            chunksize = 100,
                                            passes = 10,
                                            alpha = "auto"
                                           )

In [11]:
# Visualize the LDA Model

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds = "mmds", R = 10)
vis

/Users/theomitchell/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:228: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info  = pd.DataFrame({'saliency': saliency, 'Term': vocab, \
/Users/theomitchell/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/theomitchell/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/theomitchell/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp impo

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.210586 -0.043768       1        1  35.402947
1      0.069027  0.250089       2        1  20.806573
2     -0.147665 -0.136419       3        1  20.689211
3     -0.131251  0.086733       4        1  20.252437
0     -0.000697 -0.156634       5        1   2.848833, topic_info=            Term        Freq       Total Category  logprob  loglift
34          fire  648.000000  648.000000  Default  10.0000  10.0000
1286      region  207.000000  207.000000  Default   9.0000   9.0000
54    management   91.000000   91.000000  Default   8.0000   8.0000
62     objective  100.000000  100.000000  Default   7.0000   7.0000
115         burn  221.000000  221.000000  Default   6.0000   6.0000
...          ...         ...         ...      ...      ...      ...
139         fuel    6.904767   65.267845   Topic5  -5.0424   1.3120
573     research    5.048493   61.736999   Topic5  -5.3555   1.0545
696         item    4.350082   51.537414   Topic5  -5.5044   1.0861
67        policy    4.953241   95.975613   Topic5  -5.3745   0.5942
50          land    3.657522  183.057872   Topic5  -5.6778  -0.3548

[120 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
202       3  0.923277  advertising
203       2  0.705885       agency
203       3  0.179272       agency
203       4  0.112045       agency
4596      2  0.975684    allotment
...     ...       ...          ...
451       4  0.363846         work
198       1  0.612128         year
198       2  0.274762         year
198       3  0.076516         year
198       4  0.038258         year

[211 rows x 3 columns], R=10, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 3, 4, 1])